In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 加载预训练的BERT模型和分词器（全局范围）
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def extract_keywords():
    # 打开文件对话框，让用户选择要上传的文件
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    
    if not file_path:
        messagebox.showwarning("警告", "未选择任何文件")
        return

    try:
        # 读取用户选择的文件内容
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        
        # 将文本数据转换为适合BERT模型输入的格式
        inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=512)

        # 输入文本数据到BERT模型中，获取模型的嵌入输出
        outputs = model(inputs)

        # 提取嵌入输出
        embedding_output = outputs.last_hidden_state.numpy()[0]
        word_ids = inputs['input_ids'][0].numpy()

        # 计算词语之间的余弦相似度矩阵
        similarity_matrix = cosine_similarity(embedding_output)

        # 构建词语与索引的映射字典，过滤掉特殊符号
        special_tokens = set([tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id])
        id2word = {idx: tokenizer.convert_ids_to_tokens([word_id])[0] for idx, word_id in enumerate(word_ids) if word_id not in special_tokens}

        # 提取关键词
        keywords = []
        for i in range(len(word_ids)):
            if word_ids[i] not in special_tokens:
                word = id2word[i]
                score = np.mean(similarity_matrix[i])
                keywords.append((word, score))

        # 根据重要性分数降序排列关键词
        keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

        # 将提取的关键词结果显示在文本框中
        result_text.delete('1.0', tk.END)
        for keyword, score in keywords:
            result_text.insert(tk.END, f"{keyword}: {score:.4f}\n")
    except Exception as e:
        messagebox.showerror("错误", f"处理文件时出现问题：{e}")

# 创建主窗口
root = tk.Tk()
root.title("关键词提取工具")

# 创建上传按钮
upload_button = tk.Button(root, text="上传文件", command=extract_keywords)
upload_button.pack(pady=10)

# 创建显示结果的文本框
result_text = tk.Text(root, height=20, width=50)
result_text.pack()

# 运行主循环
root.mainloop()


C:\Users\ASUS\anaconda3\envs\TensorFlow\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np

# 加载预训练的 BERT 模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def extract_keywords():
    # 打开文件对话框，让用户选择要上传的文件
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    
    if not file_path:
        messagebox.showwarning("警告", "未选择任何文件")
        return

    try:
        # 读取用户选择的文件内容
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # 将文本数据转换为适合 BERT 模型输入的格式
        inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=512)

        # 输入文本数据到 BERT 模型中，获取模型的输出
        outputs = model(inputs)

        # 获取嵌入输出
        embedding_output = outputs.last_hidden_state.numpy()[0]

        # 获取词汇 ID 对应的词语
        word_ids = inputs['input_ids'][0].numpy()
        tokens = tokenizer.convert_ids_to_tokens(word_ids)

        # 计算嵌入输出的均值作为每个词的分数
        word_scores = np.mean(embedding_output, axis=1)

        # 组合标记和对应的分数成为关键词列表，并过滤特殊标记
        special_tokens = {tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token}
        keywords = [(token, score) for token, score in zip(tokens, word_scores) if token not in special_tokens]

        # 按照重要性分数降序排列关键词
        keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

        # 将提取的关键词结果显示在文本框中
        result_text.delete('1.0', tk.END)
        for keyword, score in keywords:
            result_text.insert(tk.END, f"{keyword}: {score:.4f}\n")
    except Exception as e:
        messagebox.showerror("错误", f"处理文件时出现问题：{e}")

# 创建主窗口
root = tk.Tk()
root.title("关键词提取工具")

# 创建上传按钮
upload_button = tk.Button(root, text="上传文件", command=extract_keywords)
upload_button.pack(pady=10)

# 创建显示结果的文本框
result_text = tk.Text(root, height=20, width=50)
result_text.pack()

# 运行主循环
root.mainloop()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w